In [ ]:
import gym
from gym import spaces
import numpy as np

class ConversationEnv(gym.Env):
    def __init__(self):
        super(ConversationEnv, self).__init__()
        # Define action and observation space
        self.action_space = spaces.Discrete(3)  # Assume three possible responses
        self.observation_space = spaces.Discrete(1)  # Simplified context state
        
        self.current_step = 0
        self.questions = ["How are you?", "What is your name?", "What is your favorite color?"]
        self.answers = ["Good, thanks!", "I am a chatbot.", "I like blue."]
        self.state = 0  # Simple state to keep track of the conversation

    def step(self, action):
        correct_answer = self.answers[self.current_step]
        if action == correct_answer:
            reward = 1  # Positive reward for correct response
        else:
            reward = -1  # Negative reward for incorrect response

        self.current_step = (self.current_step + 1) % len(self.questions)
        done = self.current_step == 0
        info = {}
        
        return self.state, reward, done, info

    def reset(self):
        self.current_step = 0
        self.state = 0
        return self.state

    def render(self, mode='human', close=False):
        print(f"Question: {self.questions[self.current_step]}")


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = []
        self.gamma = 0.95  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = tf.keras.Sequential()
        model.add(layers.Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(layers.Dense(24, activation='relu'))
        model.add(layers.Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.randint(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [ ]:
env = ConversationEnv()
agent = DQNAgent(env.observation_space.n, env.action_space.n)

episodes = 1000

for e in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, 1])
    for time in range(500):
        env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, 1])
        agent.memory.append((state, action, reward, next_state, done))
        state = next_state
        if done:
            print(f"Episode: {e+1}/{episodes}, Score: {time}, Epsilon: {agent.epsilon:.2}")
            break
        
        if len(agent.memory) > 32:
            agent.replay(32)
